In [1]:
model_checkpoint = "facebook/wav2vec2-base"
batch_size = 32

In [78]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
!apt install git-lfs

'apt' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
from transformers.utils import send_example_telemetry

send_example_telemetry("audio_classification_notebook", framework="pytorch")

In [16]:
from datasets import load_dataset, load_metric
from datasets import Dataset, DatasetDict
import os
import soundfile as sf

data_dir = "D:\\Code\\ProjectsPython\\ML_TrainingGround\\ML_Audio\\data\\speech_commands_v0.01"
categories = ["go", "no", "tree", "three"]
metric = load_metric("accuracy")

# Iterate over the categories and load the audio files
data = []
for category in categories:
    category_dir = os.path.join(data_dir, category)
    for filename in os.listdir(category_dir):
        if filename.endswith('.wav'):  # Assuming the files are WAV format
            filepath = os.path.join(category_dir, filename)
            # You can read the file here and extract features or just store the path
            data.append({'file': filepath, 'label': category})
            
dataset = Dataset.from_dict({"file": [x["file"] for x in data], "label": [x["label"] for x in data]})

# If you want to split the dataset into train and test sets, you can do that here
# For example, let's split the dataset 80-20 for train and test
train_test_split = dataset.train_test_split(test_size=0.2)

# Create a DatasetDict
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

C:\Users\ondre\AppData\Local\Temp\ipykernel_5272\3688059246.py:8: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [30]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['file', 'label'],
        num_rows: 7068
    })
    test: Dataset({
        features: ['file', 'label'],
        num_rows: 1768
    })
})

In [33]:
dataset_dict["train"][0]

{'file': 'D:\\Code\\ProjectsPython\\ML_TrainingGround\\ML_Audio\\data\\speech_commands_v0.01\\three\\0f7205ef_nohash_0.wav',
 'label': 'three'}

In [ ]:
from datasets import Audio, ClassLabel, DatasetDict
import torchaudio
from torchaudio.transforms import Resample

def preprocess_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    if sample_rate != 16000:
        resampler = Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)
    return {"array": waveform.squeeze().numpy(), "sampling_rate": 16000, "path": file_path}

# Assuming 'dataset_dict' is your DatasetDict containing 'train' and 'test' splits
# Apply the preprocessing function to the 'file' field in your dataset
for split in dataset_dict.keys():
    dataset_dict[split] = dataset_dict[split].map(lambda x: {"audio": preprocess_audio(x['file'])})

# Define the ClassLabel feature with the known labels
label_feature = ClassLabel(names=categories)

# If necessary, update the dataset to have integer labels
for split in dataset_dict.keys():
    dataset_dict[split] = dataset_dict[split].map(lambda x: {"label": label_feature.str2int(x['label'])})

# Generate id2label and label2id mappings
id2label = {str(i): label for i, label in enumerate(categories)}
label2id = {label: str(i) for i, label in enumerate(categories)}

# Now you can access the dataset as follows
example = dataset_dict["train"][0]
audio_array = example["audio"]["array"]
audio_sampling_rate = example["audio"]["sampling_rate"]
audio_path = example["audio"]["path"]
label = id2label[str(example["label"])]


In [ ]:
dataset_dict["train"].features["label"] = label_feature
label_feature

ClassLabel(names=['go', 'no', 'tree', 'three'], id=None)

In [ ]:
dataset_dict["train"][0]["audio"]["array"]

In [ ]:
id2label["3"]

'three'

## Listen to samples

In [100]:
import random
from IPython.display import Audio, display

for _ in range(5):
    rand_idx = random.randint(0, len(dataset_dict["train"])-1)
    example = dataset_dict["train"][rand_idx]
    audio = example["audio"]

    print(f'Label: {id2label[str(example["label"])]}')
    print(f'sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

Label: go
sampling rate: 16000



Label: go
sampling rate: 16000



Label: go
sampling rate: 16000



Label: go
sampling rate: 16000



Label: no
sampling rate: 16000


In [101]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

d:\Code\ProjectsPython\ML_TrainingGround\ML_Audio\.venv\lib\site-packages\transformers\configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [102]:
max_duration = 1.0  # seconds

In [111]:
print(dataset_dict["train"][0]["label"])

3


In [112]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, 
        sampling_rate=feature_extractor.sampling_rate, 
        max_length=int(feature_extractor.sampling_rate * max_duration), 
        truncation=True, 
    )
#    inputs["label"] = [int(label) for label in examples["label"]]
    return inputs

In [113]:
preprocess_function(dataset_dict['train'][:5])

{'input_values': [array([0.01924535, 0.02634905, 0.01598948, ..., 0.05298796, 0.03759659,
       0.01983732], dtype=float32), array([0.01203424, 0.00237894, 0.00244044, ..., 0.02384198, 0.03595722,
       0.04044662], dtype=float32), array([ 0.04323921,  0.07112601,  0.07978054, ..., -0.12215839,
       -0.12504323, -0.09811804], dtype=float32), array([ 0.07279099,  0.15283056,  0.23287012, ...,  0.07279099,
       -0.08728815,  0.07279099], dtype=float32), array([-0.00057058, -0.00057058, -0.00057058, ..., -0.00290543,
       -0.00348914, -0.00290543], dtype=float32)], 'label': [3, 1, 2, 3, 1]}

In [114]:
encoded_dataset = dataset_dict.map(preprocess_function, remove_columns=["audio", "file"], batched=True)
encoded_dataset

Map:   0%|          | 0/7068 [00:00<?, ? examples/s]

Map:   0%|          | 0/1768 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values'],
        num_rows: 7068
    })
    test: Dataset({
        features: ['label', 'input_values'],
        num_rows: 1768
    })
})

## Train


In [115]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint, 
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

d:\Code\ProjectsPython\ML_TrainingGround\ML_Audio\.venv\lib\site-packages\transformers\configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'projector.weight', 'classifier.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'classifier.bias', 'projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [116]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-ks",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

In [117]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [122]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"].with_format("torch"),
    eval_dataset=encoded_dataset["test"].with_format("torch"),
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics
)

In [123]:
trainer.train()

  0%|          | 0/275 [00:00<?, ?it/s]

{'loss': 1.384, 'learning_rate': 1.0714285714285714e-05, 'epoch': 0.18}
{'loss': 1.3563, 'learning_rate': 2.1428571428571428e-05, 'epoch': 0.36}
{'loss': 1.2216, 'learning_rate': 2.9757085020242918e-05, 'epoch': 0.54}
{'loss': 0.9634, 'learning_rate': 2.854251012145749e-05, 'epoch': 0.72}
{'loss': 0.7465, 'learning_rate': 2.7327935222672065e-05, 'epoch': 0.9}


  0%|          | 0/56 [00:00<?, ?it/s]

{'eval_loss': 0.5198091268539429, 'eval_accuracy': 0.9236425339366516, 'eval_runtime': 6.1317, 'eval_samples_per_second': 288.338, 'eval_steps_per_second': 9.133, 'epoch': 1.0}


d:\Code\ProjectsPython\ML_TrainingGround\ML_Audio\.venv\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


...

d:\Code\ProjectsPython\ML_TrainingGround\ML_Audio\.venv\lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.2934, 'learning_rate': 5.465587044534413e-06, 'epoch': 4.16}
{'loss': 0.3033, 'learning_rate': 4.251012145748988e-06, 'epoch': 4.34}
{'loss': 0.311, 'learning_rate': 3.0364372469635627e-06, 'epoch': 4.52}
{'loss': 0.2591, 'learning_rate': 1.8218623481781377e-06, 'epoch': 4.71}
{'loss': 0.2878, 'learning_rate': 6.072874493927125e-07, 'epoch': 4.89}


  0%|          | 0/56 [00:00<?, ?it/s]

{'eval_loss': 0.18816222250461578, 'eval_accuracy': 0.9638009049773756, 'eval_runtime': 5.8283, 'eval_samples_per_second': 303.347, 'eval_steps_per_second': 9.608, 'epoch': 4.98}
{'train_runtime': 570.0302, 'train_samples_per_second': 61.997, 'train_steps_per_second': 0.482, 'train_loss': 0.5134092265909368, 'epoch': 4.98}


TrainOutput(global_step=275, training_loss=0.5134092265909368, metrics={'train_runtime': 570.0302, 'train_samples_per_second': 61.997, 'train_steps_per_second': 0.482, 'train_loss': 0.5134092265909368, 'epoch': 4.98})

In [124]:
trainer.evaluate()

  0%|          | 0/56 [00:00<?, ?it/s]

{'eval_loss': 0.18816222250461578,
 'eval_accuracy': 0.9638009049773756,
 'eval_runtime': 5.8454,
 'eval_samples_per_second': 302.462,
 'eval_steps_per_second': 9.58,
 'epoch': 4.98}

In [125]:
trainer.push_to_hub()

events.out.tfevents.1699575655.DESKTOP-MUPP1FQ.5272.4:   0%|          | 0.00/363 [00:00<?, ?B/s]

'https://huggingface.co/Teapack1/wav2vec2-base-finetuned-ks/tree/main/'

In [ ]:
from transformers import AutoModelForAudioClassification, AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("anton-l/my-awesome-model")
model = AutoModelForAudioClassification.from_pretrained("anton-l/my-awesome-model")